In [1]:
#import basic modules
import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [6]:
#designate location of data files and growth condition. We will assume that the files are stored in this folder.
folder = '../MCMC/'
growth = 'metal'
#find names of all the files where we labeled each significant base pair.
allnames = glob.glob(folder + '*' +  growth + '*195*auto_p1percentv2_15.txt')

In [7]:
allnames

['../MCMC/iapmetaldataset_alldone_with_largeMCMC195_0_0_mut_window_auto_p1percentv2_15.txt',
 '../MCMC/aphAmetaldataset_alldone_with_largeMCMC195_0_0_mut_window_auto_p1percentv2_15.txt',
 '../MCMC/rspAmetaldataset_alldone_with_largeMCMC195_0_0_mut_window_auto_p1percentv2_15.txt',
 '../MCMC/bdcRmetaldataset_alldone_with_largeMCMC195_0_0_mut_window_auto_p1percentv2_15.txt']

In [8]:
#initialize output df with columns for the gene name, number of features for the gene,start location of
#feature, end location of feature (this will be 15 base pairs short of the true end of the region due to the
#fact that the analysis averages over 15 base pairs to find these regions), and whether the region is an 
#activator or repressor (which we determine based on if expression shift is positive or negative.)
outdf = pd.DataFrame(columns=['gene','feat_num','start','end','type'])
counter = 0
for z,name in enumerate(allnames):
    
    #load in file that says where signficant bases are.
    temp_significant = np.loadtxt(name)
    #need to automatically determine gene name. to do this we will remove any leading directory information
    #then pull the gene from the start of the file name
    noleader = name.split('/')[-1]
    gene = noleader.split(growth)[0]
    #initialize the variable 'ongoing'. this shows whether or not the current base is part of a binding site
    #or if its starting a new one.
    ongoing = 0
    TF_type = 'None'
    num_feat = 0
    #loop through only 145 base pairs (we don't go to 160 becuase the regions are 15 base pairs long.)
    for i in range(0,145):
        
        #if we are not currently part of a binding site then we do this...
        if not ongoing:
            #if we have a new significant base pair we will start to log a new binding site.
            if (temp_significant[i-1] == 0 or i == 0) and temp_significant[i] == 1:
                start = i
                ongoing = 1
                #we checked whether temp_signficant was '1' which means the effect of mutation on expression
                #is positive. This would indicate that the transcription factor type is 'repressor' 
                TF_type = 'rep'
            elif (temp_significant[i-1] == 0 or i == 0) and temp_significant[i] == -1:
                start = i
                ongoing = 1
                #we checked whether temp_signficant was '-1' which means the effect of mutation on expression
                #is negative. This would indicate that the transcription factor type is 'activator' 
                TF_type = 'act'
        elif ongoing:
            #if we are currently within an ongoing binding site we need to see whether or not the binding site
            #has ended at the current base. To do that we first check if the new base is not significant.
            if temp_significant[i] == 0:
                #next, if there is only a 1-4 base pair break in which bases are significant, the whole
                #thing is probably still part of one binding site. So we check whether or not the next 4
                #base pairs are not significant, if they are not we declare the binding site ended.
                future_sum = temp_significant[i:i+4].sum()
                #if the binding site is an activator, and none of the next four bases are also activator like
                #we end the binding site (if instead they are repressor like, in other words if
                #significant is > 0, then we can still assume the binding site has ended)
                if future_sum > -.5 and (TF_type == 'act'):
                    end = i
                    ongoing = 0
                    
                    #now that the current binding site has ended we will update the list of binding sites.
                    outdf.loc[counter,['gene','feat_num','start','end','type']] = [gene,num_feat,start,end,TF_type]
                    num_feat = num_feat + 1
                    TF_type = 'None'
                    counter = counter + 1
                #now do the same in the case that the current binding site is a repressor.
                elif future_sum < .5 and (TF_type == 'rep'):
                    end = i
                    ongoing = 0
                    outdf.loc[counter,['gene','feat_num','start','end','type']] = [gene,num_feat,start,end,TF_type]
                    num_feat = num_feat + 1
                    TF_type = 'None'
                    counter = counter + 1
                else:
                    pass
                
                    

In [10]:
#save list of all binding sites.
outdf.to_csv('../MCMC/all_regions_' + growth + '_p1percent')